In [ ]:
#| hide
from sherlock.core import *

# Sherlock Domains Python SDK

> AI agent & SDK to buy and managed domains from [Sherlock Domains](https://www.sherlockdomains.com/)

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Fewsats/sherlock-python.git
```

```sh
$ pip install sherlock-domains
```


[repo]: https://github.com/Fewsats/sherlock-python
[pypi]: https://pypi.org/project/sherlock-domains/


## How to use

Create a Sherlock instance with a private key for the agent to use. If no key is provided, a new one will be generated and saved to the default config file.

In [ ]:
s = Sherlock()
s

Sherlock(pubkey=90ba884688884277e49080712f386eebc88806efa8345ca937f75fe80950156d)

You can search for a domain and request to purchase it. Purchasing a domain requires contact information as mandated by ICANN.

In [ ]:
sr = s.search("trakwiska.com")
# Don't print the whole lists
sr['available'] = sr['available'][:1]
sr['unavailable'] = sr['unavailable'][:1]
sr

{'id': '812125a5-1358-4eaf-8589-c50248e3d4af',
 'created_at': '2025-01-14T03:36:38.352Z',
 'available': [{'name': 'trakwiska.com',
   'tld': 'com',
   'tags': [],
   'price': 1105,
   'currency': 'USD',
   'available': True}],
 'unavailable': []}

Contact information is needed for registering a new domain with the ICANN.

In [ ]:
c = Contact(**{
    "first_name": "Test",
    "last_name": "User",
    "email": "test@example.com",
    "address": "123 Test St",
    "city": "Test City",
    "state": "CA",
    "country": "US",
    "postal_code": "12345",
})

s.set_contact_information(
    cfn=c.first_name,
    cln=c.last_name,
    cem=c.email,
    cadd=c.address,
    cct=c.city,
    cst=c.state,
    cpc=c.postal_code,
    ccn=c.country
)


{'message': 'Contact information updated successfully'}

Finalizing a purchase involves a payment. Sherlock Domains currently supports two payment methods: Credit Card (`credit_card`) and the Lightning Netowrk (`lightning`).
By default the credit card method is used. In order to complete the payment: 
-  `credit_card` returns a checkout URL.
-  `lightning` returns a lightning invoice.

In [ ]:
s.request_payment_details(sr['id'], "trakwiska.com")

{'payment_method': {'checkout_url': 'https://checkout.stripe.com/c/pay/cs_live_a1uu7dX1qtgk5YH3BdJewKlRPCujUfcW0KdBWr8WLwXCuctBQkWMdE2UQx#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl',
  'lightning_invoice': None},
 'expires_at': '2025-01-14T04:06:39.531Z'}

You can now use the checkout URL to complete the purchase and the domain will be registered to your agent.

You can view the domains you own with:

In [ ]:
s.domains()

[{'id': 'd9b2cc30-c15d-44b9-9d39-5d33da504484',
  'domain_name': 'h402.org',
  'created_at': '2024-12-28T18:58:49.899Z',
  'expires_at': '2024-12-31T18:58:42Z',
  'auto_renew': False,
  'locked': True,
  'private': True,
  'nameservers': [],
  'status': 'active'}]

Below is a list of all the tools that the client offers to manage the domains and purchases.

In [ ]:
s.as_tools().map(lambda t: t.__name__)

(#10) ['_me','_set_contact_information','_get_contact_information','_search','_request_payment_details','_domains','_dns_records','_create_dns_record','_update_dns_record','_delete_dns_record']

## AI agents

We will show how to enable your AI assistant to handle payments using [Claudette](https://claudette.answer.ai), Answer.ai convenient wrapper for Claude. You'll need to export your `ANTHROPIC_API_KEY`.

In [ ]:
from claudette import Chat, models

In [ ]:
model = models[1]; model

'claude-3-5-sonnet-20240620'

Create a Sherlock instance with a public & private key for the agent to use.

Sherlock supports returning all the tools with `s.as_tools()`.

In [ ]:
sp = 'You are a helpful assistant that has access to a domain purchase API.'
chat = Chat(model, sp=sp, tools=s.as_tools())

pr = f"Search if domain 'the-favourite-game.com' is available? If it is request a purchase and process the payment using credit card method."
r = chat.toolloop(pr, trace_func=print)
r

Message(id='msg_014pq4tF62joNFyY7HK6a197', content=[TextBlock(text="Certainly! I'll search for the domain 'the-favourite-game.com' and if it's available, I'll proceed with the purchase request using the credit card payment method. Let's start with the search.", type='text'), ToolUseBlock(id='toolu_01P6nMob3NuohiyNYDvYjmde', input={'q': 'the-favourite-game.com'}, name='_search', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 2409; Out: 104; Cache create: 0; Cache read: 0; Total: 2513)
Message(id='msg_01NcViW2skMwxQFMFMKr3g5n', content=[TextBlock(text="Great news! The domain 'the-favourite-game.com' is available. The search results show that it can be purchased for 1105 cents USD, which is equivalent to $11.05.\n\nNow that we know the domain is available, let's proceed with the purchase request using the credit card payment method. However, before we can do that, we need to make sure that the 

Great! The purchase request has been processed successfully. Here's a summary of what happened:

1. We searched for the domain 'the-favourite-game.com' and found it was available for &#36;11.05.
2. We confirmed that the contact information was already set up.
3. We submitted a purchase request for the domain using the credit card payment method.

The system has provided a checkout URL for you to complete the payment. To finalize the purchase, you'll need to visit this URL:

https://checkout.stripe.com/c/pay/cs_live_a1o69y0arBV5CHaLPQ4lusmVfLqGJDjAkyhNeTVT89Ri3LBbjQQjE3iuT8#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl

Please note that this checkout URL will expire on 2025-01-14T04:06:49.681Z. Make sure to complete the payment before this time to secure your domain.

Is there anything else you'd like me to help you with regarding this domain purchase or any other domain-related tasks?

<details>

- id: `msg_01TodMupqc2jGtgHzKnz4d1J`
- content: `[{'text': "Great! The purchase request has been processed successfully. Here's a summary of what happened:\n\n1. We searched for the domain 'the-favourite-game.com' and found it was available for $11.05.\n2. We confirmed that the contact information was already set up.\n3. We submitted a purchase request for the domain using the credit card payment method.\n\nThe system has provided a checkout URL for you to complete the payment. To finalize the purchase, you'll need to visit this URL:\n\nhttps://checkout.stripe.com/c/pay/cs_live_a1o69y0arBV5CHaLPQ4lusmVfLqGJDjAkyhNeTVT89Ri3LBbjQQjE3iuT8#fidkdWxOYHwnPyd1blppbHNgWjA0S3VzXDdBbTFNVlJzfDVRQVQ2dVdBTnJTSH1QMGs2dHRsanJMbkY0PTxKbUtRaWowT2NwMGM8RlVBbGRqSWo3UFYwcVdqR3F9N2BtM2ZTPXc1Z3dQXGc2NTVPYVVSQkM8bycpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl\n\nPlease note that this checkout URL will expire on 2025-01-14T04:06:49.681Z. Make sure to complete the payment before this time to secure your domain.\n\nIs there anything else you'd like me to help you with regarding this domain purchase or any other domain-related tasks?", 'type': 'text'}]`
- model: `claude-3-5-sonnet-20240620`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 3361, 'output_tokens': 477}`

</details>